## Byte-Encoding Representation

This is a subword tokenization of a vocabulary. An example of how this would work and be useful at test time from Sennrich et al. is as follows:

> vocab = ‘low’, ‘lowest’, ‘newer’, ‘wider’

> OOV = 'lower' --> 'low_', 'er_'

This is much more valuable than a symbol for a generic unknown word (UNK). It would not be surpirising if given a large enough corpus a model would be able to intepret `lower` correctly.

To build this representation, an iterative algorithm can be used to link together the most common segments, starting with character pairs. Below is the pseudo code provided by the original authors with a few changes.

In [1]:
import re
import sys
import collections

This algorithm is iterative. The author provides a more optimized implementation that batches and avoids recomputation. It starts bottom up, getting the bigram counts for every pair of symbols. These symbols will startout as characters but in later steps be common strings.

In [2]:
def get_stats(vocab):
    """For the given merge step, get the bigram counts, for every pair of symbols (strings).

    For each word and its freq in the corpus, add the number of instances for each of its subword parts.
    """
    pairs = collections.defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    return pairs

Next, given the bi-counts of a vocabulary, merge the vocabulary to remove repetitions of this bigram.

In [3]:
def merge_vocab(pair, v_in):
    v_out = {}
    bigram_pattern = re.escape(' '.join(pair))
    print(bigram_pattern)
    p = re.compile(r'(?<!\S)' + bigram_pattern + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    print(v_out)
    return v_out

In [4]:
def byte_pair(vocab, num_merges=5):
    for i in range(num_merges):
        pairs = get_stats(vocab)
        best = max(pairs, key=pairs.get)
        if pairs[best] < 2:
            print('no pair has frequency > 1. Stopping\n')
            break
        vocab = merge_vocab(best, vocab)
    print(vocab)
vocab = {
  'l o w </w>' : 5, 
  'f a r t h e s t </w>' : 5,
  'n e w e r </w>': 5,
  'w i d e r </w>': 5 }
byte_pair(vocab)

e\ r
{'l o w </w>': 5, 'f a r t h e s t </w>': 5, 'n e w er </w>': 5, 'w i d er </w>': 5}
er\ </w>
{'l o w </w>': 5, 'f a r t h e s t </w>': 5, 'n e w er</w>': 5, 'w i d er</w>': 5}
l\ o
{'lo w </w>': 5, 'f a r t h e s t </w>': 5, 'n e w er</w>': 5, 'w i d er</w>': 5}
lo\ w
{'low </w>': 5, 'f a r t h e s t </w>': 5, 'n e w er</w>': 5, 'w i d er</w>': 5}
low\ </w>
{'low</w>': 5, 'f a r t h e s t </w>': 5, 'n e w er</w>': 5, 'w i d er</w>': 5}
{'low</w>': 5, 'f a r t h e s t </w>': 5, 'n e w er</w>': 5, 'w i d er</w>': 5}


TODO(cjlovering): show how OOV is built up.